In [19]:
# 1. Tạo class Dataset để fetch dữ liệu
# 2. Khảo sát phân phối của dữ liệu và chuẩn hóa trọng số để cân chỉnh mức độ phụ thuộc
# 3. Cắt nhỏ dataset ra làm nhiều phần để train dần và vẫn đảm bảo phân bố của dữ liệu
# 4. Tạo Dataloader cho tập train và valid
# 5. Xây dựng mô hình, hàm thất thoát, bộ tối ưu và chọn siêu tham số
# 6. Tạo cái j đó để theo dõi đánh giá và lưu lại quá trình huấn luyện mô hình
# 7. Huấn luyện mô hình
# 8. Đánh giá mô hình

## 0. Import configuration

In [37]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import yaml
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import os
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix

import time
import copy

with open("config/config.yaml", "r") as f:
    conf = yaml.safe_load(f)

x_train_path = conf['dataset']['x_train']
y_train_path = conf['dataset']['y_train']

x_valid_path = conf['dataset']['x_valid']
y_valid_path = conf['dataset']['y_valid']

batch_size = conf['parameter']['batch_size']
learning_rate = conf['parameter']['learning_rate']
epochs = conf['parameter']['epochs']
num_chunk = conf['chunk']

## 1. Create Dataset class

In [38]:
# 1. Create dataset class
class HAM10KDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform

        self.class_cols = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
        # Create a mapping from class name to numerical label
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.class_cols)}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # get image
        img_name = self.labels_df.iloc[idx]['image'] + '.jpg'
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # get numerical label
        encode_label = self.labels_df.iloc[idx][self.class_cols].values
        string_label = self.class_cols[encode_label.argmax()]
        numerical_label = self.class_to_idx[string_label]

        if self.transform:
            image = self.transform(image)

        return image, numerical_label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_set = HAM10KDataset(
    csv_path=y_train_path,   # CSV
    img_dir=x_train_path,    # thư mục ảnh
    transform=transform
)

valid_set = HAM10KDataset(
    csv_path=y_valid_path,
    img_dir=x_valid_path,
    transform=transform
)

print(f"Kích thước tập train: {len(train_set)}")
print(f"Kích thước tập validation: {len(valid_set)}")

Kích thước tập train: 10015
Kích thước tập validation: 193


## 2. Examine data distribution and Calculate Classes' Weights

In [39]:
y_train_df = train_set.labels_df
class_cols = train_set.class_cols
class_counts = {cls: 0 for cls in class_cols}

for i, row in y_train_df.iterrows():
    for cls in class_cols:
        if row[cls] == 1:
            class_counts[cls] += 1
            break

print(class_counts)

total = len(y_train_df)
num_classes = len(class_cols)

class_weights = []
for cls_name in class_cols:
    count = class_counts[cls_name]
    weight = total / (num_classes * count)
    class_weights.append(weight)

class_weights = np.array(class_weights)
class_weights = class_weights / class_weights.sum() # chuẩn hóa để để đưa vào hàm Focal loss
class_weights

{'MEL': 1113, 'NV': 6705, 'BCC': 514, 'AKIEC': 327, 'BKL': 1099, 'DF': 115, 'VASC': 142}


array([0.03958192, 0.00657042, 0.08570948, 0.13472377, 0.04008614,
       0.3830841 , 0.31024417])

## 3. Chunking Dataset

In [40]:
# Chunking dataset into 10 small subset with the same distribution
labels = np.argmax(train_set.labels_df[train_set.class_cols].values, axis=1)
indices = np.arange(len(train_set))

# Đảm bảo mỗi tập con đều vẫn giữ nguyên phân phối như tập lớn
skf = StratifiedKFold(
    n_splits=num_chunk,
    shuffle=True,
    random_state=1
)

k_chunk_indices = []
for fold_id, (whatisthis, chunk_indices) in enumerate(skf.split(indices, labels)):
    print(f"Chunk {fold_id+1} with {len(chunk_indices)} samples")
    count = Counter(labels[chunk_indices])
    count = {int(k): int(v) for k, v in count.items()}
    print(dict(sorted(count.items())))
    k_chunk_indices.append(chunk_indices)

Chunk 1 with 1002 samples
{0: 111, 1: 671, 2: 52, 3: 32, 4: 110, 5: 12, 6: 14}
Chunk 2 with 1002 samples
{0: 111, 1: 671, 2: 52, 3: 32, 4: 110, 5: 12, 6: 14}
Chunk 3 with 1002 samples
{0: 111, 1: 671, 2: 52, 3: 33, 4: 110, 5: 11, 6: 14}
Chunk 4 with 1002 samples
{0: 111, 1: 671, 2: 51, 3: 33, 4: 110, 5: 11, 6: 15}
Chunk 5 with 1002 samples
{0: 111, 1: 671, 2: 51, 3: 33, 4: 110, 5: 11, 6: 15}
Chunk 6 with 1001 samples
{0: 112, 1: 670, 2: 51, 3: 33, 4: 110, 5: 11, 6: 14}
Chunk 7 with 1001 samples
{0: 112, 1: 670, 2: 51, 3: 33, 4: 110, 5: 11, 6: 14}
Chunk 8 with 1001 samples
{0: 112, 1: 670, 2: 51, 3: 33, 4: 109, 5: 12, 6: 14}
Chunk 9 with 1001 samples
{0: 111, 1: 670, 2: 51, 3: 33, 4: 110, 5: 12, 6: 14}
Chunk 10 with 1001 samples
{0: 111, 1: 670, 2: 52, 3: 32, 4: 110, 5: 12, 6: 14}


## 4. Create DataLoader for train and valid

In [41]:
valid_loader = DataLoader(valid_set, batch_size = batch_size, shuffle=True)

chunk_loaders = []

for i, chunk_indices in enumerate(k_chunk_indices):
    chunk_subset = Subset(
        train_set,
        chunk_indices
    )

    loader = DataLoader(
        chunk_subset,
        batch_size = batch_size,
        shuffle=True
    )

    chunk_loaders.append(loader)

chunk_loaders

## 5. Define model

### 5.1 Build model

In [42]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()

        # Depthwise convolution
        self.depthwise = nn.Sequential(
            nn.Conv2d(
                in_channels,
                in_channels,
                kernel_size=3,
                stride=stride,
                padding=1,
                groups=in_channels,
                bias=False
            ),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True)
        )

        # Pointwise convolution
        self.pointwise = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=1,
                stride=1,
                padding=0,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

In [43]:
class MobileNetV1(nn.Module):
    def __init__(self, num_classes=7, width_mult=1.0):
        super().__init__()

        def c(ch):
            return int(ch * width_mult)

        self.features = nn.Sequential(
            # Conv 1
            nn.Conv2d(3, c(32), kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(c(32)),
            nn.ReLU(inplace=True),

            # Depthwise blocks
            DepthwiseSeparableConv(c(32),  c(64),  stride=1),
            DepthwiseSeparableConv(c(64),  c(128), stride=2),
            DepthwiseSeparableConv(c(128), c(128), stride=1),
            DepthwiseSeparableConv(c(128), c(256), stride=2),
            DepthwiseSeparableConv(c(256), c(256), stride=1),
            DepthwiseSeparableConv(c(256), c(512), stride=2),

            # 5 × (512 → 512)
            DepthwiseSeparableConv(c(512), c(512), stride=1),
            DepthwiseSeparableConv(c(512), c(512), stride=1),
            DepthwiseSeparableConv(c(512), c(512), stride=1),
            DepthwiseSeparableConv(c(512), c(512), stride=1),
            DepthwiseSeparableConv(c(512), c(512), stride=1),

            DepthwiseSeparableConv(c(512), c(1024), stride=2),
            DepthwiseSeparableConv(c(1024), c(1024), stride=1),
        )

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(c(1024), num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

### 5.2 Loss function

In [44]:
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha, gamma=2):
        super(FocalLoss, self).__init__()
        self.register_buffer(
            "alpha", torch.tensor(alpha, dtype=torch.float32)
        )
        self.gamma = gamma

    def forward(self, logits, targets):
        ce = F.cross_entropy(logits, targets, reduction='none')
        pt = torch.exp(-ce)
        alpha_t = self.alpha.to(logits.device)[targets]
        loss = alpha_t * (1 - pt) ** self.gamma * ce
        return loss.mean()

print("FocalLoss class defined.")

FocalLoss class defined.


### 5.3 Model operations

In [86]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    runnning_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for imgs, labels in dataloader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        runnning_loss += loss.item() * imgs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = runnning_loss / total_samples
    epoch_acc = correct_predictions / total_samples
    return epoch_loss, epoch_acc

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_prediction = 0
    total_samples = 0

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs.data, 1)

            total_samples += labels.size(0)
            correct_prediction += (preds == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_acc = correct_prediction / total_samples
    return epoch_loss, epoch_acc

print("Training and validation epoch functions defined.")

Training and validation epoch functions defined.


## 6. Creat model checkpoint save/loader + Metrics log

### 6.1 Checkpoints save/loader

In [115]:
from datetime import datetime
import csv, os, json
from sklearn.metrics import f1_score, recall_score

def save_checkpoint(
        model,
        optimizer,
        epoch,
        metrics,
        save_dir
):
    ckpt = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "metrics": metrics,
        "timestamp": datetime.now().isoformat()
    }

    path = f"{save_dir}/checkpoint/epoch_{epoch:02d}.pt"
    torch.save(ckpt, path)

def load_checkpoint(dir_path, model, optimizer, device):
    ckpt_files = [
        f for f in os.listdir(dir_path)
        if f.endswith(".pt")
    ]

    if len(ckpt_files) != 0:
        
        #Sort theo epoch
        ckpt_files.sort()
        latest_ckpt = ckpt_files[-1]

        ckpt_path = os.path.join(dir_path, latest_ckpt)
        ckpt = torch.load(ckpt_path, map_location=device)

        model.load_state_dict(ckpt["model_state"])
        optimizer.load_state_dict(ckpt["optimizer_state"])

        start_epoch = ckpt["epoch"] + 1
        metrics = ckpt.get("metrics")
        
        print(f"[INFO] Resume training from {latest_ckpt}")
        print(f"       Continue with epoch {start_epoch} with valid acc {metrics['val_acc']:.4f}")
    else:
        print(f"[INFO] No checkpoints found ! Consider start training some..")
        model = None
        optimizer = None
        start_epoch = 0
        best_acc = 0

    return model, optimizer, start_epoch, metrics

### 6.2 Metrics log

In [84]:
def log_to_csv(path, row):
    write_header = not os.path.exists(path)

    with open(path, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if write_header:
            writer.writeheader()
        writer.writerow(row)

def log_to_json(path, entry):
    if os.path.exists(path):
        with open(path,"r") as f:
            logs = json.load(f)
    else:
        logs = []

    entry["timestamp"] = datetime.now().isoformat()
    logs.append(entry)

    with open(path, "w") as f:
        json.dump(logs, f, indent=2)

## 7. Train mô hình thôi

### 7.1 Khởi tạo tham số training

In [117]:
# Thiết lập mô hình
device = "cuda" if torch.cuda.is_available() else "cpu"

resume_ckpt_path = r"experiments\mobilenetv1_ham10k\checkpoint"

model = MobileNetV1(num_classes=7).to(device)
criterion = FocalLoss(alpha=class_weights, gamma=2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

model, optimizer, start_epoch, metrics = load_checkpoint(resume_ckpt_path, model, optimizer, device)

best_model_wts = copy.deepcopy(model.state_dict())

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
best_acc = metrics['val_acc']

# Log theo dõi thông số
save_dir = "experiments/mobilenetv1_ham10k"
os.makedirs(f"{save_dir}/checkpoint", exist_ok=True)
os.makedirs(f"{save_dir}/logs", exist_ok=True)

csv_log_path = f"{save_dir}/logs/train_log.csv"
json_log_path = f"{save_dir}/logs/train_log.json"



[INFO] Resume training from epoch_03.pt
       Continue with epoch 4 with valid acc 0.6580


### 7.2 Training...

In [118]:
print("Starting training ...")

for epoch in range(start_epoch-1, epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for i, train_loader in enumerate(chunk_loaders):
        epoch_start_time = time.time()

        # Training phase
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        # Validation phase
        val_loss, val_acc = validate_epoch(model, valid_loader, criterion, device)        
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        # Print epoch statistics
        print(f'     Chunk {i+1}/{num_chunk} - ' \
            f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - ' \
            f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f} - ' \
            f'Time: {(time.time() - epoch_start_time):.2f}s')

        # kiếm chỗ để bổ sung thêm hàm log để theo dõi metrics và lưu thông số mô hình lại
        csv_row = {
            "epoch": epoch + 1,
            "chunk_id": i + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        }
        log_to_csv(csv_log_path, csv_row)

        json_entry = {
            "epoch": epoch + 1,
            "chunk_id": i + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        }
        log_to_json(json_log_path, json_entry)

        # Save the best model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), f'{save_dir}/best_mobilenetv1_model.pth')
            print(f"  --> Best model saved with Validation Accuracy: {best_acc:.4f}")

    # Lưu checkpoint tại mỗi epoch
    save_checkpoint(model=model, optimizer=optimizer, epoch=epoch+1, metrics=json_entry, save_dir=save_dir)

print("\nTraining complete.")
print(f"Best Validation Accuracy: {best_acc:.4f}")

# Load the best model weights
model.load_state_dict(best_model_wts)

Starting training ...
Epoch 4/10
     Chunk 1/10 - Train Loss: 0.0234, Train Acc: 0.5190 - Val Loss: 0.0324, Val Acc: 0.4715 - Time: 121.72s
     Chunk 2/10 - Train Loss: 0.0231, Train Acc: 0.5509 - Val Loss: 0.0293, Val Acc: 0.5544 - Time: 126.88s
     Chunk 3/10 - Train Loss: 0.0224, Train Acc: 0.5200 - Val Loss: 0.0262, Val Acc: 0.5492 - Time: 126.10s
     Chunk 4/10 - Train Loss: 0.0242, Train Acc: 0.4970 - Val Loss: 0.0242, Val Acc: 0.6114 - Time: 136.09s
     Chunk 5/10 - Train Loss: 0.0208, Train Acc: 0.5070 - Val Loss: 0.0263, Val Acc: 0.6166 - Time: 150.41s
     Chunk 6/10 - Train Loss: 0.0215, Train Acc: 0.5315 - Val Loss: 0.0237, Val Acc: 0.5492 - Time: 129.34s
     Chunk 7/10 - Train Loss: 0.0223, Train Acc: 0.4875 - Val Loss: 0.0217, Val Acc: 0.6062 - Time: 200.19s
     Chunk 8/10 - Train Loss: 0.0227, Train Acc: 0.5465 - Val Loss: 0.0248, Val Acc: 0.5596 - Time: 168.77s
     Chunk 9/10 - Train Loss: 0.0219, Train Acc: 0.5495 - Val Loss: 0.0309, Val Acc: 0.6891 - Time: 151

KeyboardInterrupt: 

## 8. Đánh giá mô hình
Confusion matrix, accuracy, recall, precision, f1-score,...

### 8.1 Suy luận trên tập Test

### 8.2 Visualize kết quả